In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data/ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 294480 entries, 0 to 294479
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       294480 non-null  int64 
 1   timestamp     294480 non-null  object
 2   group         294480 non-null  object
 3   landing_page  294480 non-null  object
 4   converted     294480 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 11.2+ MB


In [4]:
df

,user_id,timestamp,group,landing_page,converted
0,851104,11:48.6,control,old_page,0
1,804228,01:45.2,control,old_page,0
2,661590,55:06.2,treatment,new_page,0
3,853541,28:03.1,treatment,new_page,0
4,864975,52:26.2,control,old_page,1
...,...,...,...,...,...
294475,734608,45:03.4,control,old_page,0
294476,697314,20:29.0,control,old_page,0
294477,715931,40:24.5,treatment,new_page,0
294478,759899,20:29.0,treatment,new_page,0


In [5]:
# some user id are repeated
df.user_id.unique().shape[0] == df.user_id.shape[0]

False

In [6]:
df = df.drop_duplicates(subset= 'user_id', keep=False)

In [7]:
df.describe()

,user_id,converted
count,286690.000000,286690.000000
mean,788036.057184,0.119450
std,91239.396095,0.324317
min,630000.000000,0.000000
25%,709036.250000,0.000000
50%,788059.500000,0.000000
75%,866998.750000,0.000000
max,945999.000000,1.000000


In [8]:
group_count = df.groupby(['landing_page', 'converted'])['landing_page'].count().reset_index(name='Count')
group_count = pd.crosstab(index=group_count['landing_page'], columns=group_count['converted'], 
            values=group_count['Count'], aggfunc=sum, margins=True)
group_count

converted,0,1,All
landing_page,,,
new_page,126372,17025,143397
old_page,126073,17220,143293
All,252445,34245,286690


In [9]:
groupped = group_count.div(group_count['All'], axis=0)
groupped

converted,0,1,All
landing_page,,,
new_page,0.881274,0.118726,1.0
old_page,0.879827,0.120173,1.0
All,0.880550,0.119450,1.0


It looks like the treatment has worse result

In [10]:
df_control = df[df['group'] == 'control']
df_treatment = df[df['group'] == 'treatment']

### Normality Assumption

H0 : The assumption of normality is provided.     
H1 : The assumption of normality is not provided.

alpha - 0.05

In [11]:
from scipy.stats import shapiro # Shapiro-Wilk test
from scipy.stats import kstest # Kolmogorov–Smirnov test

In [12]:
test_statistic, p_value = shapiro(df['converted'])
if p_value > 0.05:
    print('The assumption of normality is provided')
else:
    print('The assumption of normality is not provided')

The assumption of normality is not provided


/Users/vittoria/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [13]:
result = kstest(df['converted'], "norm")
if result.pvalue > 0.05:
    print('The assumption of normality is provided')
else:
    print('The assumption of normality is not provided')

The assumption of normality is not provided


### Variance Homogeneity     
H0 : Variances are homogeneous    
H1 : Variances are not homogeneous 

alpha - 0.05

In [14]:
from scipy.stats import levene
# Levene test for equal variances.

In [15]:
test_stat, pvalue = levene(df_control['converted'], df_treatment['converted'])
if p_value > 0.05:
    print('Variances are homogeneous') 
else:
    print('Variances are not homogeneous')

Variances are not homogeneous


### Mann-Whitney U test    
(The U test (non-parametric test) is performed if normality and homogeneity of variance are not provided)

In terms of click rate to the desired page:      
H0 : There is no significant difference between the two groups.    
H1 : There is a difference between the two groups.

alpha - 0.05

In [16]:
from scipy.stats.mstats import mannwhitneyu

In [17]:
statistic, p_value = mannwhitneyu(df_control['converted'], df_treatment['converted'])
if p_value > 0.05:
    print('There is no significant difference between the two groups in terms of click rate to the desired page.') 
else:
    print('There is a difference between the two groups in terms of click rate to the desired page.')


There is no significant difference between the two groups in terms of click rate to the desired page.


### Conclusion     
The treatment sample result is worse than the control sample, but this has not been proven statistically.
I believe the experiment should take longer to make a decision.